In [ ]:
# 用前须知

## xtdata提供和MiniQmt的交互接口，本质是和MiniQmt建立连接，由MiniQmt处理行情数据请求，再把结果回传返回到python层。使用的行情服务器以及能获取到的行情数据和MiniQmt是一致的，要检查数据或者切换连接时直接操作MiniQmt即可。

## 对于数据获取接口，使用时需要先确保MiniQmt已有所需要的数据，如果不足可以通过补充数据接口补充，再调用数据获取接口获取。

## 对于订阅接口，直接设置数据回调，数据到来时会由回调返回。订阅接收到的数据一般会保存下来，同种数据不需要再单独补充。

# 代码讲解

# 从本地python导入xtquant库，如果出现报错则说明安装失败
from xtquant import xtdata
import time

xtquant文档地址：http://dict.thinktrader.net/nativeApi/start_now.html


In [ ]:

# 设定一个标的列表
code_list = ["TL00.IF"]
# 判断当前主力合约
main_contract = [xtdata.get_main_contract(code) for code in code_list]
print(f"当前主力合约: {main_contract}")

# 设定获取数据的周期
period = "tick"


***** xtdata连接成功 *****
服务信息: {'tag': 'sp3', 'version': '1.0'}
服务地址: 127.0.0.1:58610
数据路径: C:\国金证券QMT交易端\bin.x64/../userdata_mini/datadir
设置xtdata.enable_hello = False可隐藏此消息

当前主力合约: ['TL2503.IF']


In [ ]:

# 下载标的行情数据
if 1:
    ## 为了方便用户进行数据管理，xtquant的大部分历史数据都是以压缩形式存储在本地的
    ## 比如行情数据，需要通过download_history_data下载，财务数据需要通过
    ## 所以在取历史数据之前，我们需要调用数据下载接口，将数据下载到本地
    for contract in main_contract:
        xtdata.download_history_data(contract, period=period, incrementally=True) # 增量下载行情数据（开高低收,等等）到本地
    
    # xtdata.download_financial_data(code_list) # 下载财务数据到本地
    # xtdata.download_sector_data() # 下载板块数据到本地
    # # 更多数据的下载方式可以通过数据字典查询


{'TL00.IF': Empty DataFrame
Columns: [time, lastPrice, open, high, low, lastClose, amount, volume, pvolume, tickvol, stockStatus, openInt, lastSettlementPrice, askPrice, bidPrice, askVol, bidVol, settlementPrice, transactionNum, pe]
Index: []}


In [10]:
# 读取本地历史行情数据
history_data = xtdata.get_market_data_ex([], ['TL2503.IF'], period=period, count=-1)
history_data['TL2503.IF'].head()

<bound method NDFrame.head of                          time  lastPrice  open  high   low  lastClose  amount  \
20240617092900  1718587740100     107.60  0.00  0.00  0.00     107.60    0.00   
20240617092903  1718587743579     107.61  0.01  0.01  0.01     107.61    4.42   
20240623180133  1719136893022     107.62  0.02  0.02  0.02     107.62   17.52   
20240623180136  1719136896501     107.63  0.03  0.03  0.03     107.63   39.29   
20240623180139  1719136899055     107.64  0.04  0.04  0.04     107.64   68.90   
...                       ...        ...   ...   ...   ...        ...     ...   
19700101080000              0       0.00  0.00  0.00  0.00       0.00    0.00   
19700101080000              0       0.00  0.00  0.00  0.00       0.00    0.00   
19700101080000              0       0.00  0.00  0.00  0.00       0.00    0.00   
19700101080000              0       0.00  0.00  0.00  0.00       0.00    0.00   
19700101080000              0       0.00  0.00  0.00  0.00       0.00    0.00  

In [ ]:
import pandas as pd

# Set display options to avoid omissions
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

history_data['TL2503.IF'][:100]

,time,lastPrice,open,high,low,lastClose,amount,volume,pvolume,tickvol,stockStatus,openInt,lastSettlementPrice,askPrice,bidPrice,askVol,bidVol,settlementPrice,transactionNum,pe
20240617092900,1718587740100,107.60,0.00,0.00,0.00,107.60,0.00,0,0,0,0,0,107.60,"[107.42, 0.0, 0.0, 0.0, 0.0]","[106.92, 0.0, 0.0, 0.0, 0.0]","[5, 0, 0, 0, 0]","[5, 0, 0, 0, 0]",0.00,0,0.00000
20240617092903,1718587743579,107.61,0.01,0.01,0.01,107.61,4.42,7,1,27303403460034669,1,1,107.61,"[106.38, 106.39, 106.4, 106.41000000000001, 10...","[106.34, 106.33, 106.32, 106.30999999999999, 1...","[12, 1, 1, 1, 1]","[10, 1, 1, 1, 1]",0.01,1,0.00001
20240623180133,1719136893022,107.62,0.02,0.02,0.02,107.62,17.52,14,2,27303403460034670,2,2,107.62,"[106.35000000000001, 106.36000000000001, 106.3...","[106.35000000000001, 106.34, 106.33, 106.32, 1...","[15, 2, 2, 2, 2]","[16, 2, 2, 2, 2]",0.02,2,0.00002
20240623180136,1719136896501,107.63,0.03,0.03,0.03,107.63,39.29,21,3,-9196068633394741137,3,3,107.63,"[106.36, 106.37, 106.38000000000001, 106.39000...","[106.38, 106.36999999999999, 106.3599999999999...","[20, 3, 3, 3, 3]","[21, 3, 3, 3, 3]",0.03,3,0.00003
20240623180139,1719136899055,107.64,0.04,0.04,0.04,107.64,68.90,28,4,27303403460034670,4,4,107.64,"[106.38, 106.39, 106.4, 106.41000000000001, 10...","[106.37, 106.36, 106.35, 106.33999999999999, 1...","[27, 4, 4, 4, 4]","[-652, 4, 4, 4, 4]",0.04,4,0.00004
20240623180142,1719136902532,107.65,0.05,0.05,0.05,107.65,105.08,34,5,-9196068633394741137,5,5,107.65,"[107.02000000000001, 107.03000000000002, 107.0...","[106.38000000000001, 106.37, 106.36, 106.35, 1...","[23, 5, 5, 5, 5]","[363, 5, 5, 5, 5]",0.05,5,0.00005
20240623180143,1719136903086,107.66,0.06,0.06,0.06,107.66,143.94,41,6,-9196068633394741136,6,6,107.66,"[106.39, 106.4, 106.41000000000001, 106.420000...","[106.39, 106.38, 106.36999999999999, 106.35999...","[30, 6, 6, 6, 6]","[361, 6, 6, 6, 6]",0.06,6,0.00006
20240623180146,1719136906467,107.67,0.07,0.07,0.07,107.67,182.19,44,7,-9196068633394741135,7,7,107.67,"[106.4, 106.41000000000001, 106.42000000000002...","[106.72, 106.71, 106.69999999999999, 106.68999...","[33, 7, 7, 7, 7]","[354, 7, 7, 7, 7]",0.07,7,0.00007
20240623180148,1719136908972,107.68,0.08,0.08,0.08,107.68,243.31,51,8,-9196068633394741134,8,8,107.68,"[106.57000000000001, 106.58000000000001, 106.5...","[106.41000000000001, 106.4, 106.39, 106.38, 10...","[39, 8, 8, 8, 8]","[352, 8, 8, 8, 8]",0.08,8,0.00008
20240623180152,1719136912353,107.69,0.09,0.09,0.09,107.69,313.24,58,9,-9196068633394741133,9,9,107.69,"[106.42, 106.43, 106.44000000000001, 106.45000...","[106.42, 106.41, 106.39999999999999, 106.38999...","[46, 9, 9, 9, 9]","[359, 9, 9, 9, 9]",0.09,9,0.00009


In [32]:
history_data['TL2503.IF'].to_csv(r'C:\Users\tonym\Desktop\TL2503.IF.csv')